# Adding a new game to the framework - FirstLast

- The players should engage in a turn-based conversation about a predefined topic. 
- Player A starts with an utterance whose first and last token must start with a `predefined letter`, say d. 
- Player B must then reply with an utterance whose first and last token must be the `next one in the alphabet` (here, an e). <br> <br>
And so on, for n turns **(where each turn is comprised by an utterance from A and an utterance from B)**.<br> <br>
- If an utterance does not conform to these rules (i.e. it is incorrect), the players lose the game. <br>
- `move rule:` If an utterance does not start with 'I SAY: ' (i.e., it is invalid), the game is immediately aborted. <br>
- If all utterances up to turn n are valid and correct, the game is successful.

For instance, if the topic is **birds**, the initial letter is `h` and the number of turns is 2, this would be a successful game:

- Hi! I love birds, but it's hard to identify them. I need help. `(h: hi / help)`
- I know what you mean. I can try to help, please describe it. `(i: I/ it)`
- Just a moment... Ok, it's blue but looks like an Eurasian jay. `(j: just / jay)`
- Kick in more details, otherwise I don't know. `(k: kick / know)`

Each turn, we need to check two aspects: 
- **MOVE_RULE**: Does the utterance start with 'I SAY'?
- **GAME_RULE**: Do the first/last tokens start with `predefined_letter`?

# Adding a new game:
We will need at least the following components:
- **Game resources:** all data, prompt templates and text files that are necessary to create instances of a game and to group these instances into experiments.
- **Instances:** JSON containing the configuration of each instance, grouped into experiments. 
    - This must be done by a script named `instancegenerator.py`, with a class that inherits from `GameInstanceGenerator`.
- **Game Master:** Controls and enforces MOVE/GAME Rules, inheriting from GameMaster. 
    - This must be implemented in a file `master.py`.
- **Players:** Defines the programatic behaviour and any other attributes of a player, inheriting from `Player`.
    - This can be implemented in a file named `players.py`.
- **Game Benchmark:** a class that realises the game, inheriting from `GameBenchmark`. 
    - This can also live in the file `master.py`.

To define an **Episode**, we have to instantiate the initial promopts and define 3 more parameters:
- Topic
- Letter
- Number of Turns

# Defining prompts with game rules
- In the prompt template we can define variables that can be replaced later.
- Prompts have to be adjusted for Player A and B, based on their roles.

In [ ]:
with open(path / 'resources' / 'initial_prompts' / 'initial_prompt_a.template', 'w') as file:
    file.write(
        "Let's play a game. You must have a conversation about $topic with your partner. Your first turn must start and end with words that begin with the letter $letter. The reply of your partner must be similar, with the letter that comes after $letter in the alphabet. Then it's your turn again with the next letter, and so on. You'll do it for $nturns turns. Always start your utterance with I SAY: and then give your answer. If you break the rules, you lose."
    ) 

with open(path / 'resources' / 'initial_prompts' /  'initial_prompt_b.template', 'w') as file:
    file.write(
        "Let's play a game. You must have a conversation about $topic with your partner. Their first turn must start and end with words that begin with the letter $letter. Your reply must be similar, with the letter that comes after $letter in the alphabet. Then it's their turn again with the next letter, and so on. You'll do it for $nturns turns. Always start your utterance with I SAY: and then give your answer. If you break the rules, you lose."
    ) 